In [ ]:
# Let us see how to build NN algorithm
# How to finetune (hyperparameters like no. of neurons, no. of hidden layers), activation function
# let us see how to select best optimized parameters for the given dataset.
# Importing the necessary packages
import pandas as pd
import numpy as np
import keras # library built on top of Tensar flow. It is used to build NN as well as to optimize hyperparameter.
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load pima indians dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.csv", delimiter=",")
# dellimiter=The string is used to separate values. By default, this is any whitespace. The delimiter to use for parsing the content of txt a file.

In [ ]:
X = dataset[:,0:8]
y = dataset[:,8]

In [ ]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

In [ ]:
pd.DataFrame(X_standardized).describe()

,0,1,2,3,4,5,6,7
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-8.789266e-17,-9.251859e-18,1.503427e-17,8.211024e-17,-4.972874e-17,2.960595e-15,1.905883e-15,2.081668e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


#### ****Tuning of Hyperparameters :- ****

1.   Batch Size - at a time how many data point to send to the network
2.   Epochs - how many times we are sending data to the network



In [ ]:
# create NN and find best batch size and best epchs
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential # In NN we are going to add components (i.e. hidden layers) one by one in sequential manner
from keras.layers import Dense # the no. of neurons
from keras.wrappers.scikit_learn import KerasClassifier
#from keras.optimizers import Adam
from keras.optimizers import adam_v2
from tensorflow.keras.optimizers import Adam # Adam - Adaptive Momentum - is an optimizer

In [ ]:
# create model
def create_model():
    model = Sequential()
    #below is the first hidden layer
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu')) 
    #Dense is a NN, 12 - no. of neurons, i/p - 8 columns, kernel_initializer=uniform - bias or weights can be taken from uniform distribution.
    #below is the second hidden layer
    model.add(Dense(8,kernel_initializer='uniform', activation='relu'))
    #below is the output layer
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    adam=Adam(learning_rate=0.01) # Adam is the optimizer from stochastic Gradient descent variant, lr - learning rate - eta
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    # using binary cross entropy find accuracy of model
    return model

In [ ]:
# Create the model
model = KerasClassifier(build_fn = create_model,verbose = 0) # verbose=0 means don't print any information on screen
# Define the grid search parameters
batch_size = [10,20,40] # out of these 3 which is best value that we need to find using accuracy
# model will be built using 10 batch size once, then 20, then 40
epochs = [10,50,100,150] # makes all combinations all batch size and epochs
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10) # by default it takes 5 K folds, verbose =10: displays 10 lines of epochs 1/12 for 10 times
grid_result = grid.fit(X_standardized,y)
# batch size=3, epoch=4. 3*4=12, 12*5 folds = 60 models will be built

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START batch_size=10, epochs=10...................................
[CV 1/5; 1/12] END ....batch_size=10, epochs=10;, score=0.721 total time=   3.3s
[CV 2/5; 1/12] START batch_size=10, epochs=10...................................
[CV 2/5; 1/12] END ....batch_size=10, epochs=10;, score=0.682 total time=   2.4s
[CV 3/5; 1/12] START batch_size=10, epochs=10...................................
[CV 3/5; 1/12] END ....batch_size=10, epochs=10;, score=0.753 total time=   2.4s
[CV 4/5; 1/12] START batch_size=10, epochs=10...................................
[CV 4/5; 1/12] END ....batch_size=10, epochs=10;, score=0.810 total time=   3.7s
[CV 5/5; 1/12] START batch_size=10, epochs=10...................................
[CV 5/5; 1/12] END ....batch_size=10, epochs=10;, score=0.771 total time=   2.1s
[CV 1/5; 2/12] START batch_size=10, epochs=50...................................
[CV 1/5; 2/12] END ....batch_size=10, epochs=50;

In [ ]:
# Summarize the results - Let us find out which combination gave us the best result
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7669977068901062, using {'batch_size': 40, 'epochs': 10}
0.747508704662323,0.04379143621571187 with: {'batch_size': 10, 'epochs': 10}
0.7526186227798461,0.019930475578770337 with: {'batch_size': 10, 'epochs': 50}
0.7474662661552429,0.03174703095991465 with: {'batch_size': 10, 'epochs': 100}
0.7357440114021301,0.027459917876398805 with: {'batch_size': 10, 'epochs': 150}
0.7540361762046814,0.05586029108777838 with: {'batch_size': 20, 'epochs': 10}
0.7617944121360779,0.03552371667931185 with: {'batch_size': 20, 'epochs': 50}
0.7591885328292847,0.026677887389491627 with: {'batch_size': 20, 'epochs': 100}
0.7408963680267334,0.021434852141519207 with: {'batch_size': 20, 'epochs': 150}
0.7669977068901062,0.03813163306045334 with: {'batch_size': 40, 'epochs': 10}
0.7644003033638,0.033759266223978866 with: {'batch_size': 40, 'epochs': 50}
0.7462015151977539,0.045043624564131475 with: {'batch_size': 40, 'epochs': 100}
0.7410576462745666,0.0632847077764714 with: {'batch_size': 40, 'epoch

In [ ]:
params


[{'dropout_rate': 0.0, 'learning_rate': 0.001},
 {'dropout_rate': 0.0, 'learning_rate': 0.01},
 {'dropout_rate': 0.0, 'learning_rate': 0.1},
 {'dropout_rate': 0.1, 'learning_rate': 0.001},
 {'dropout_rate': 0.1, 'learning_rate': 0.01},
 {'dropout_rate': 0.1, 'learning_rate': 0.1},
 {'dropout_rate': 0.2, 'learning_rate': 0.001},
 {'dropout_rate': 0.2, 'learning_rate': 0.01},
 {'dropout_rate': 0.2, 'learning_rate': 0.1}]

#### Tuning of Hyperparameters:- Learning rate and Drop out rate

In [ ]:
from keras.layers import Dropout
# drop out rate - to control the accuracy. To overcome overfit of model randomly remove connection of some neurons
# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(12,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters: Learning rate and dropout rate are the hyperparameters we want to test
 
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]# drop 0%,10%, 20% neurons

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10) # total 9*5 default folds=45 models will be built. (Lrate 3 values, Dropout rate =3, 3*3=9,)
grid_result = grid.fit(X_standardized,y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.766 total time=   1.5s
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.708 total time=   1.4s
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.760 total time=   1.4s
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.804 total time=   1.5s
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.752 total time=   1.8s
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 1/5; 

In [ ]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7709107995033264, using {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.7578643560409546,0.03079243193129651 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
0.7617859363555908,0.03415134632059508 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
0.7553179025650024,0.03555767787583205 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
0.7527034997940063,0.04904802817584358 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.7618368625640869,0.04792013648315022 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.764391815662384,0.03181082685734347 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.7358119010925293,0.06510971104113665 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.7709107995033264,0.03360267367686723 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.764391815662384,0.037387986114403096 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


#### Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [ ]:
# Defining the model

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(12,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.01)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model
    # In machine learning, Loss function is used to find error or deviation in the learning process. 
    # Keras requires loss function during model compilation process. https://www.tutorialspoint.com/keras/keras_model_compilation.htm

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

# Define the grid search parameters
activation_function = ['softmax','relu','tanh','linear'] # find which activation function is best out of these
init = ['uniform','normal','zero'] # Weight initializers from where the weights has to be sampled randomly. Uniform distribution, normal distribution and zero distribution

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)



Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 1/5; 1/12] END activation_function=softmax, init=uniform;, score=0.721 total time=   3.0s
[CV 2/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 2/5; 1/12] END activation_function=softmax, init=uniform;, score=0.695 total time=   3.8s
[CV 3/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 3/5; 1/12] END activation_function=softmax, init=uniform;, score=0.747 total time=   3.8s
[CV 4/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 4/5; 1/12] END activation_function=softmax, init=uniform;, score=0.843 total time=   3.7s
[CV 5/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 5/5; 1/12] END activation_function=softmax, init=uniform;, score=0.758 total time=   5.2s
[CV 1/5; 2/12] START activation_function=softmax, init=normal...

In [ ]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7722179770469666, using {'activation_function': 'linear', 'init': 'uniform'}
0.7527289867401123,0.050229315452837535 with: {'activation_function': 'softmax', 'init': 'uniform'}
0.7500976204872132,0.03966489317151111 with: {'activation_function': 'softmax', 'init': 'normal'}
0.747483241558075,0.04052559938787095 with: {'activation_function': 'softmax', 'init': 'zero'}
0.763127064704895,0.055822389263087066 with: {'activation_function': 'relu', 'init': 'uniform'}
0.7579152941703796,0.052471402582764184 with: {'activation_function': 'relu', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'relu', 'init': 'zero'}
0.7709192752838134,0.03871262465053147 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.7708938002586365,0.03641916859309435 with: {'activation_function': 'tanh', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'tanh', 'init': 'zero'}
0.7722179770469666,0.03679523849381246 with: {'activ

#### Tuning of Hyperparameter :-Number of Neurons in hidden layer

In [ ]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = 'uniform',activation = 'linear'))
    model.add(Dropout(0.2))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'linear'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.01)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

# Define the grid search parameters

neuron1 = [4,8,16]# in first hidden layer use 4,8,16 neurons
neuron2 = [2,4,8]# in second hidden layer use 2,4,8 neurons

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................


[CV 1/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.779 total time=   4.0s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................


[CV 2/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.721 total time=   3.9s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................


[CV 3/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.773 total time=   3.8s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................


[CV 4/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.837 total time=   3.9s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................


...

[CV 1/5; 9/9] END ........neuron1=16, neuron2=8;, score=0.773 total time=   4.1s
[CV 2/5; 9/9] START neuron1=16, neuron2=8.......................................


[CV 2/5; 9/9] END ........neuron1=16, neuron2=8;, score=0.734 total time=   3.3s
[CV 3/5; 9/9] START neuron1=16, neuron2=8.......................................


[CV 3/5; 9/9] END ........neuron1=16, neuron2=8;, score=0.766 total time=   3.0s
[CV 4/5; 9/9] START neuron1=16, neuron2=8.......................................


[CV 4/5; 9/9] END ........neuron1=16, neuron2=8;, score=0.830 total time=   2.8s
[CV 5/5; 9/9] START neuron1=16, neuron2=8.......................................


[CV 5/5; 9/9] END ........neuron1=16, neuron2=8;, score=0.758 total time=   3.8s


In [ ]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7722179770469666, using {'neuron1': 16, 'neuron2': 4}
0.7721925139427185,0.03811601562336581 with: {'neuron1': 4, 'neuron2': 2}
0.7682964086532593,0.03703779136171633 with: {'neuron1': 4, 'neuron2': 4}
0.7669892191886902,0.035367753979321855 with: {'neuron1': 4, 'neuron2': 8}
0.7722094893455506,0.031112063764460036 with: {'neuron1': 8, 'neuron2': 2}
0.7682964086532593,0.03970953669469943 with: {'neuron1': 8, 'neuron2': 4}
0.7709107875823975,0.037403206253799934 with: {'neuron1': 8, 'neuron2': 8}
0.7695866227149963,0.03115039431472802 with: {'neuron1': 16, 'neuron2': 2}
0.7722179770469666,0.035154315407367395 with: {'neuron1': 16, 'neuron2': 4}
0.7721925139427185,0.03180650819148932 with: {'neuron1': 16, 'neuron2': 8}


# #### Training model with optimum values of Hyperparameters

In [ ]:
#skip this
from sklearn.metrics import classification_report, accuracy_score

# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 8,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 16,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.01) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Fitting the model

model.fit(X_standardized,y)

# Predicting using trained model

y_predict = model.predict(X_standardized)

# Printing the metrics
print(accuracy_score(y,y_predict))

24/24 [==============================] - 0s 1ms/step
0.7747395833333334


# Hyperparameters all at once


The hyperparameter optimization was carried out by taking 2 hyperparameters at once. We may have missed the best values. The performance can be further improved by finding the optimum values of hyperparameters all at once given by the code snippet below.
#### This process is computationally expensive.

In [ ]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0)

# Define the grid search parameters

batch_size = [10,20,40]
epochs = [10,50,100]
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']
neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Streaming output truncated to the last 5000 lines.
[CV 1/5; 9/8748] END activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=16, neuron2=8;, score=0.649 total time=   1.6s
[CV 2/5; 9/8748] START activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=16, neuron2=8
[CV 2/5; 9/8748] END activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=16, neuron2=8;, score=0.584 total time=   2.4s
[CV 3/5; 9/8748] START activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=16, neuron2=8
[CV 3/5; 9/8748] END activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs=10, init=uniform, learning_rate=0.001, neuron1=16, neuron2=8;, score=0.630 total time=   2.0s
[CV 4/5; 9/8748] START activation_function=softmax, batch_size=10, dropout_rate=0.0, epochs